In [12]:
import optuna
import numpy as np
import time

# Schwefel function
def schwefel(trial):
    # Определяем количество признаков
    dim = 10
    # Генерируем значения признаков
    x = [trial.suggest_float(f"x{i}", -500, 500) for i in range(dim)]
    
    # Вычисляем значение функции Швефеля
    return 418.9829 * dim - sum([xi * np.sin(np.sqrt(abs(xi))) for xi in x])

# Словарь для хранения результатов всех итераций
results = []

# Запуск оптимизации
def optimize_schwefel():
    start_time = time.time()  # Засекаем время начала
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
    
    # Внутренняя функция для сохранения результата на каждой итерации
    def callback(study, trial):
        # Сохраняем результаты каждой итерации
        results.append({
            'trial_number': trial.number,
            'value': trial.value,
            'params': trial.params,
            'time': time.time() - start_time  # Засекаем время выполнения каждой итерации
        })

    study.optimize(schwefel, n_trials=50, callbacks=[callback])  # 1000 итераций

    # Находим оптимальный результат
    best_trial = study.best_trial
    best_result = None
    for result in results:
        if result['trial_number'] == best_trial.number:
            best_result = result
            break

    # Итоговый вывод
    print(f"Оптимальная итерация: {best_result['trial_number']}")
    print(f"Время до оптимальной итерации: {best_result['time']:.2f} секунд")
    print(f"Значение оптимальной функции: {best_result['value']}")
    print(f"Оптимальные значения признаков: {best_result['params']}")
    print(f"Всего итераций: {len(results)}")

# Запуск оптимизации
optimize_schwefel()


Оптимальная итерация: 47
Время до оптимальной итерации: 1.90 секунд
Значение оптимальной функции: 1870.02965694243
Оптимальные значения признаков: {'x0': 409.6666912717867, 'x1': -290.8009742977373, 'x2': -276.98105240018674, 'x3': 411.50620491237225, 'x4': 52.65007130456506, 'x5': 191.6638039710508, 'x6': 349.59764101098347, 'x7': 359.4180698429753, 'x8': 409.7907882703502, 'x9': 437.1236204777814}
Всего итераций: 50


In [2]:
import optuna
import numpy as np
import time
import pandas as pd
import logging

# Отключаем вывод Optuna
optuna.logging.set_verbosity(optuna.logging.CRITICAL)

# Устанавливаем уровень логирования Python на ERROR
logging.getLogger().setLevel(logging.ERROR)

# Функция для Schwefel
def schwefel(trial):
    dim = 10
    x = [trial.suggest_float(f"x{i}", -500, 500) for i in range(dim)]
    A = 418.9829
    return A * dim - sum([xi * np.sin(np.sqrt(abs(xi))) for xi in x])

# Запуск оптимизации одного прогона
def optimize_schwefel():
    start_time = time.time()
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))

    study.optimize(schwefel, n_trials=1000, show_progress_bar=False)  # 100 итераций на прогон

    best_trial = study.best_trial
    best_result = {
        'Значение оптимальной функции': best_trial.value,
        'Оптимальная итерация': best_trial.number,
        'Время до оптимальной итерации, сек': time.time() - start_time
    }
    return best_result

# Запуск внешнего цикла
def run_optimization(num_runs):
    csv_filename = 'opt_schwefel.csv'
    results = []

    for run in range(num_runs):
        best_result = optimize_schwefel()
        results.append(best_result)
        df_result = pd.DataFrame([best_result])
        df_result.to_csv(csv_filename, mode='a', header=not bool(run), index=False)

# Внешний цикл с количеством прогонов (настраиваемое значение)
run_optimization(30)
